In [253]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time

In [268]:
df_full= pd.read_csv('simplefullrun.csv')

In [276]:
df_full['draft_year'].min()

1993

In [308]:
df = df_full[(df_full['draft_year'] == 1993)]

In [255]:
# df= full_df[full_df['draft_year'].isin([2022,2021])]

KeyError: "None of [Int64Index([2022, 2022, 2022, 2022, 2022, 2022, 2022, 2022, 2022, 2022,\n            ...\n            1993, 1993, 1993, 1993, 1993, 1993, 1993, 1993, 1993, 1993],\n           dtype='int64', length=7576)] are in the [columns]"

In [302]:
df.head()

,draft_year,draft_round,draft_pick,draft_team,draft_name,draft_name_url,draft_position,draft_age,draft_college,draft_college_stats_url
7128,1994,1,1,CIN,Dan Wilkinson,https://www.pro-football-reference.com/players...,DT,21.0,Ohio St.,http://www.sports-reference.com/cfb/players/da...
7129,1994,1,2,IND,Marshall Faulk,https://www.pro-football-reference.com/players...,RB,21.0,San Diego St.,http://www.sports-reference.com/cfb/players/ma...
7130,1994,1,3,WAS,Heath Shuler,https://www.pro-football-reference.com/players...,QB,22.0,Tennessee,http://www.sports-reference.com/cfb/players/he...
7131,1994,1,4,NWE,Willie McGinest,https://www.pro-football-reference.com/players...,DE,22.0,USC,NaN
7132,1994,1,5,IND,Trev Alberts,https://www.pro-football-reference.com/players...,LB,24.0,Nebraska,http://www.sports-reference.com/cfb/players/tr...


In [303]:
df.shape

(224, 10)

In [260]:
# df.drop(df.index[:7500], inplace=True)

In [309]:
url=pd.Series(df['draft_name_url'])

In [285]:
len(url)

1192

In [286]:
type(url)

pandas.core.series.Series

In [287]:
url.isnull().sum()

153

In [145]:
for index, value in url.iteritems():
    print(f"Index: {index}, Value: {value}")

Index: 7500, Value: https://www.pro-football-reference.com/players/B/BoykDe20.htm
Index: 7501, Value: https://www.pro-football-reference.com/players/D/DaviSc21.htm
Index: 7502, Value: https://www.pro-football-reference.com/players/L/LyonMi00.htm
Index: 7503, Value: https://www.pro-football-reference.com/players/H/HutcPa20.htm
Index: 7504, Value: https://www.pro-football-reference.com/players/M/McKeRi20.htm
Index: 7505, Value: https://www.pro-football-reference.com/players/B/BonnMe20.htm
Index: 7506, Value: https://www.pro-football-reference.com/players/M/MorrDa20.htm
Index: 7507, Value: https://www.pro-football-reference.com/players/W/WatsTi20.htm
Index: 7508, Value: https://www.pro-football-reference.com/players/E/EtheCa20.htm
Index: 7509, Value: https://www.pro-football-reference.com/players/B/BradCh20.htm
Index: 7510, Value: nan
Index: 7511, Value: https://www.pro-football-reference.com/players/W/WychFr00.htm
Index: 7512, Value: https://www.pro-football-reference.com/players/C/CastE

In [208]:
url[0]

'https://www.pro-football-reference.com/players/W/WalkTr03.htm'

In [169]:
    if pd.isnull(loop_url):
        name_value = None
        birthdate_value = None
        birthplace_value = None
        height_value = None
        weight_value = None
    else:

IndentationError: expected an indented block (1610286114.py, line 7)

In [310]:
%%time
# Create an empty dataframe with the desired columns
df_test = pd.DataFrame(columns=['name', 'pic_url','bday', 'bplace', 'hs','height', 'weight'])

# addon = 'https://webcache.googleusercontent.com/search?q=cache:'
for loop_url in url:
    name_value = None
    pic_value = None
    birthdate_value = None
    birthplace_value = None
    hs_value = None
    height_value = None
    weight_value = None
    
    if not pd.isna(loop_url):
        r = requests.get(loop_url)
        page = BeautifulSoup(r.content, 'html.parser')
        
        # finding the text block we need
        text = page.find_all(text=re.compile("birth"))
        
        # Join the elements of the list into a single string
        text = "".join(text)

        # Use the re module to extract the text within quotes
        result = re.findall(r'"([^"]*)"', text)

        # Create a dataframe from the result
        df = pd.DataFrame(result, columns=['Data'])
        
        # Find the index of the row that contains the value 'birthdate'
        try:
            name_index = df[df['Data'] == 'Person'].index[0]
            name_value = df.loc[name_index + 2, 'Data']
        except IndexError:
            name_value = None
        try:
            pic_index = df[df['Data'] == 'contentUrl'].index[0]
            pic_value = df.loc[pic_index + 1, 'Data']
        except:
            pic_value = None
        try:
            birthdate_index = df[df['Data'] == 'birthDate'].index[0]
            birthdate_value = df.loc[birthdate_index + 1, 'Data']
        except IndexError:
            birthdate_value = None
        
        try:
            birthplace_index = df[df['Data'] == 'birthPlace'].index[0]
            birthplace_value = df.loc[birthplace_index + 1, 'Data']
        except IndexError:
            birthplace_value = None
            
        try:
            links = page.find_all("a", href=lambda href: href and href.startswith("/schools/high_schools.cgi?"))
            names = ''
            for link in links:
                names = names + link.text + ','
            hs_value= names[:-1]
        except IndexError:
                hs_value = None
        
        try:
            height_index = df[df['Data'] == 'height'].index[0]
            height_value = df.loc[height_index + 4, 'Data']
        except IndexError:
            height_value = None
        
        try:
            weight_index = df[df['Data'] == 'weight'].index[0]
            weight_value = df.loc[weight_index + 4, 'Data']
        except IndexError:
            weight_value = None
    
    print(name_value)
    # Store the variables in a dictionary
    data = {
    'name': [name_value],
    'pic_url':[pic_value],
    'bday': [birthdate_value],
    'bplace': [birthplace_value],
    'hs':[hs_value],
    'height': [height_value],
    'weight': [weight_value]
    }

    # Append the data to the dataframe
    df_test = df_test.append(pd.DataFrame(data), ignore_index=True)
    
    # Wait for 4 seconds
    time.sleep(3.4)
df_test.to_csv("FullDraftData97.csv", index=False)

Drew Bledsoe


<timed exec>:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


Rick Mirer


<timed exec>:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


Garrison Hearst


...

None
Jon Kirksey


<timed exec>:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


Trent Green


<timed exec>:88: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


None
None
CPU times: total: 29.2 s
Wall time: 13min 46s


In [289]:
df_test.isnull().sum()

name        32
pic_url    294
bday        32
bplace      32
hs          32
height      32
weight      32
dtype: int64

In [264]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   name     6168 non-null   object
 1   pic_url  3979 non-null   object
 2   bday     6168 non-null   object
 3   bplace   5843 non-null   object
 4   hs       6173 non-null   object
 5   height   6161 non-null   object
 6   weight   6161 non-null   object
dtypes: object(7)
memory usage: 355.4+ KB


In [234]:
data

{'name': ['Aidan Hutchinson'],
 'pic_url': ['https://www.pro-football-reference.com/req/20180910/images/headshots/HutcAi00_2022.jpg'],
 'bday': ['2000-08-09'],
 'bplace': ['MI'],
 'height': ['6-7'],
 'weight': ['265 lbs']}

In [307]:
df_test.head(20)

,name,pic_url,bday,bplace,hs,height,weight
0,Dan Wilkinson,https://www.pro-football-reference.com/req/201...,1973-03-13,"Dayton, OH, USA","Paul L. Dunbar,OH",6-4,340 lbs
1,Marshall Faulk,https://www.pro-football-reference.com/req/201...,1973-02-26,"New Orleans, LA, USA","George Washington Carver,LA",5-10,211 lbs
2,Heath Shuler,https://www.pro-football-reference.com/req/201...,1971-12-31,"Bryson City, NC, USA","Swain Co.,NC",6-2,216 lbs
3,Willie McGinest,https://www.pro-football-reference.com/req/201...,1971-12-11,"Long Beach, CA, USA","Long Beach Polytechnic,CA",6-5,268 lbs
4,Trev Alberts,https://www.pro-football-reference.com/req/201...,1970-08-08,"Cedar Falls, IA, USA","U. of Northern Iowa,IA",6-4,245 lbs
5,Trent Dilfer,https://www.pro-football-reference.com/req/201...,1972-03-13,"Aptos, CA, USA","Aptos,CA",6-4,225 lbs
6,Bryant Young,https://www.pro-football-reference.com/req/201...,1972-01-27,"Chicago Heights, IL, USA","Bloom,IL",6-3,291 lbs
7,Sam Adams,https://www.pro-football-reference.com/req/201...,1973-06-13,"Houston, TX, USA","Cypress Creek,TX",6-3,350 lbs
8,Antonio Langham,https://www.pro-football-reference.com/req/201...,1972-07-31,"Town Creek, AL, USA","Hazlewood,AL",6-0,181 lbs
9,Jamir Miller,https://www.pro-football-reference.com/req/201...,1973-11-19,"Philadelphia, PA, USA","El Cerrito,CA",6-5,252 lbs


In [311]:
df_test.to_csv("1993.csv", index=False)

In [247]:
df_test['pic_url'][1]

'https://www.pro-football-reference.com/req/20180910/images/headshots/HutcAi00_2022.jpg'

In [265]:
df_test.to_csv("FullDraftData.csv", index=False)

In [267]:
df_test.iloc[-1,:]

name       Antonio Banks
pic_url             None
bday          1973-03-12
bplace     Ivor, VA, USA
hs            Warwick,VA
height              5-10
weight           203 lbs
Name: 6496, dtype: object